# MCF Instances

## Formulations of MCNF Problems

Source: [Input Format for Multicommodity Flow Problems](ftp://ftp.cs.wisc.edu/math-prog/cpo-dataset/networks/multicommodity/Jones_Lustig_Farv/format.tex)

The JL format for MCNF problems defines these different ways to formulate a problem:
- **origin-destination problem (ODP)** - The commodity is defined as a product $k$, that travels between a specific *origin* $s$ and a specifig *destination* $t$. The commodity is then a triple $$\kappa = (k, s, t)$$
- **destination specific problem (DSP)** - The commodity is defined as a product that travels to a specific destination from multiple origins, or vice versa, from a specific origin to multiple destinations. The commodity is then a tuple $$\kappa = (k, t) \quad \text{or} \quad \kappa = (k, s)$$
- **product specific problem (PSP)** - The commodity is defined as a product that must travel through the network from multiple origins to multiple destinations. The commodity is then the singleton $$\kappa = k$$.

Problem instances for the Linear Multicomodity Flow Problems are taken from
[Multicommodity Problem Instances](http://groups.di.unipi.it/optimize/Data/MMCF.html#MNetGen)

In [92]:
import numpy as np
from pathlib import Path
from dataclasses import dataclass

## Input Format Overview## Input Format Overview

### Common

A problem instance description consist of 4 files:
- **Node file** (``*.nod``) for general information
- **Arc file** (``*.arc``) describing links
- **Supply/demand file** (``*.sup`` or ``*.od`` (origin-destination problems))
- **Mutual capacity file** (``*.mut``)
The node file contains four integers int the following order:




### Node File

The format of the node file is usually the same for all instances (with the exception of whitespaces).

The file contains 4 integers with the following meanings.

```
number of products
number of nodes
number of links
number of bundled links
```



### Link File

#### MNETGEN

One line of the link file contains numbers separated by whitespace.
The meaning of the numbers for a given links is specified in the following order:

```
arcname fromnode tonode commodity cost individual_capacity mutual_capacity_ptr
```

Arc name is an integer between 1 and the number of arcs, that is necessary to distinguish between multiple instances of an arc (i, j) for the same commodity, that are permitted.

The mutual capacity pointer is an ``integer`` specifying the instace of the bundled link found in the *mutual capacity file*.
*NOTE: **0** means no pointer is specified.*

The number ``-1`` in a column is used to specify that:
    - the information in the column is to be replicated over all possible instances
    - the information is not applicable to the problem
The meaning is derived from the context.

In mnetgen case ``-1`` means:
    - no individual capacity in the ``individual_capacity`` field, meaning the amount of product is bounded only by the upper bound
    
    
#### PDS

One line of the link file contains numbers separated by whitespace.
The meaning of the numbers for a given links is specified in the following order:

```
arcname fromnode tonode commodity cost individual_capacity mutual_capacity_ptr
```

In pds case ``-1`` means:
    - no individual capacity in the ``individual_capacity`` field, meaning the amount of product is bounded only by the upper bound (or unbounded)
 
#### JLF


One line of the link file contains numbers separated by whitespace.
The meaning of the numbers for a given links is specified in the following order:

```
fromnode tonode commodity cost individual_capacity origin destination mutual_capacity_ptr
```



### Supply/Demand file

#### MNETGEN

One line of the link file contains numbers separated by whitespace.

```
node commodity supply/demand
```

#### PDS

One line of the link file contains numbers separated by whitespace.

```
node commodity supply/demand
```

Supply is expressed as a positive integer, demand as a negative integer.


#### JLF

```
origin destination commodity flow
```
If flow is present ``-1`` in origin means demand, in destination supply.



### Mutual capacity file

The format of the node file is mostly the same for all instances.

One line of the link file contains numbers separated by whitespace.

```
mutual_capacity_ptr mutual_capacity
```

In [93]:
# Path to data directory
instances_path = Path("example/")

assert instances_path.exists()

In [94]:
@dataclass
class InstanceInfo:
    """ Information about the instance. """

    products_no : int
    nodes_no : int
    links_no : int
    bundled_links_no : int
    

def read_nod(nod_file):
    content = None
    with open(nod_file, 'r') as fin:
        content = fin.read()
                
    products_no, nodes_no, links_no, bundled_links_no, *_ = map(int, content.split())
    
    return InstanceInfo(products_no = products_no,
                        nodes_no = nodes_no,
                        links_no = links_no,
                        bundled_links_no = bundled_links_no)

instance_info = read_nod(nod_file)
print(instance_info)

InstanceInfo(products_no=92, nodes_no=30, links_no=150, bundled_links_no=150)


In [95]:
from collections import defaultdict

ARC_FIELD_TYPES = {
    'arcname' :np.uint32,
    'fromnode' : np.int32,
    'tonode' : np.int32,
    'commodity' : np.int32,
    'cost' :  np.float64,
    'origin': np.int32,
    'destination': np.int32,
    'individual_capacity' : np.float64,
    'mutual_ptr' : np.uint32
}
    
    
def read_arc(arc_file, instance_format):
    # TODO: Use data from node file to instantate
    
    fields = None
    if instance_format in ['mnetgen', 'pds', 'planar', 'grid']:
        fields = ('arcname', 'fromnode', 'tonode', 'commodity','cost', 'individual_capacity', 'mutual_ptr')
    elif instance_format == 'jlf':
        fields = ('fromnode', 'tonode', 'commodity', 'cost', 'individual_capacity', 'origin', 'destination', 'mutual_ptr')
    else:
        raise NotImplementedError("read_arc: Not implemented!")

    data = defaultdict(list)
    with open(arc_file, 'r') as fin:
        for line in fin:
            for field, value in zip(fields, line.split()):
                data[field].append(ARC_FIELD_TYPES[field](value))
        
        for field in data:
            data[field] = np.array(data[field], dtype=ARC_FIELD_TYPES[field])

    return data


In [107]:
SUP_FIELD_TYPES = {
    'node' : np.int32,
    'commodity' : np.int32,
    'origin': np.int32,
    'destination': np.int32,
    'flow' : np.float64
}

def read_sup(sup_file, instance_format):
    fields = None
    if instance_format in ['mnetgen', 'pds', 'planar', 'grid']:
        fields = ('node', 'commodity', 'flow')
    elif instance_format == 'jlf':
        fields = ('origin', 'destination', 'commodity', 'flow')
    else:
        raise NotImplementedError("read_sup: Not implemented!")

    data = defaultdict(list)
    with open(sup_file, 'r') as fin:
        for line in fin:
            for field, value in zip(fields, line.split()):
                data[field].append(SUP_FIELD_TYPES[field](value))
                
        for field in data:
            data[field] = np.array(data[field], dtype=SUP_FIELD_TYPES[field])
            
    return data

In [97]:
MUT_FIELD_TYPES = {
    'mutual_ptr' : np.int32,
    'capacity' : np.float64,
}

def read_mut(mut_file, instance_format):
    fields = ('mutual_ptr', 'capacity')
    if instance_format not in ['mnetgen', 'pds', 'jlf', 'planar', 'grid']:
        raise NotImplementedError("read_arc: Not implemented!")
            
    data = defaultdict(list)
    with open(mut_file, 'r') as fin:
        for line in fin:
            for field, value in zip(fields, line.split()):
                data[field].append(MUT_FIELD_TYPES[field](value))
                
        for field in data:
            data[field] = np.array(data[field], dtype=MUT_FIELD_TYPES[field])
            
    return data

In [109]:
instance_format = "planar"
nod_file = instances_path / (instance_format + ".nod")
arc_file = instances_path / (instance_format + ".arc")
sup_file = instances_path / (instance_format + ".sup")
mut_file = instances_path / (instance_format + ".mut")

nodinfo = read_nod(nod_file)
arcinfo = read_arc(arc_file, instance_format)
supinfo = read_sup(sup_file, instance_format)
mutinfo = read_mut(mut_file, instance_format)

if True:
    print(nodinfo)
    print(arcinfo)
    print(supinfo)
    print(mutinfo)

InstanceInfo(products_no=92, nodes_no=30, links_no=150, bundled_links_no=150)
defaultdict(<class 'list'>, {'arcname': array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150], dtype=uint32), 'fromnode': array([ 1, 14,  1, 1